In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset


 94% 312M/330M [00:02<00:00, 138MB/s]
100% 330M/330M [00:02<00:00, 133MB/s]


In [ ]:
!unzip -qq jangedoo/utkface-new.zip

unzip:  cannot find or open jangedoo/utkface-new.zip, jangedoo/utkface-new.zip.zip or jangedoo/utkface-new.zip.ZIP.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
cd /content/drive/MyDrive/DeepLearningTasks/FaceMaskDetection

In [ ]:
batch_size = 4
epochs = 10
lr = 0.0001
width = height = 128

In [ ]:
data_generator = ImageDataGenerator(
    rescale = 1/.255,
    horizontal_flip=True
)

train_data = data_generator.flow_from_directory(
    "/content/Face Mask Dataset/Train",
    target_size = (width, height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True,
)

val_data = data_generator.flow_from_directory(
    "/content/Face Mask Dataset/Validation",
    target_size = (width, height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True,
)

print(np.bincount(train_data.labels))
print(np.bincount(val_data.labels))

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
[5000 5000]
[400 400]


In [ ]:
model = tf.keras.models.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
])

In [ ]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                             metrics=['accuracy'])

In [ ]:
model.fit(train_data,
          # steps_per_epoch=train_data.samples/batch_size,
          validation_data=val_data,
          # validation_steps=val_data.samples/batch_size,
          epochs=epochs,
          )

Epoch 1/10
2500/2500 [==============================] - 58s 23ms/step - loss: 0.1291 - accuracy: 0.9669 - val_loss: 0.0831 - val_accuracy: 0.9762
Epoch 2/10
2500/2500 [==============================] - 56s 22ms/step - loss: 0.0588 - accuracy: 0.9830 - val_loss: 0.0572 - val_accuracy: 0.9812
Epoch 3/10
2500/2500 [==============================] - 56s 22ms/step - loss: 0.0395 - accuracy: 0.9879 - val_loss: 0.0628 - val_accuracy: 0.9825
Epoch 4/10
2500/2500 [==============================] - 56s 22ms/step - loss: 0.0342 - accuracy: 0.9904 - val_loss: 0.0583 - val_accuracy: 0.9862
Epoch 5/10
2500/2500 [==============================] - 56s 22ms/step - loss: 0.0374 - accuracy: 0.9905 - val_loss: 0.0372 - val_accuracy: 0.9900
Epoch 6/10
2500/2500 [==============================] - 55s 22ms/step - loss: 0.0331 - accuracy: 0.9907 - val_loss: 0.1338 - val_accuracy: 0.9725
Epoch 7/10
2500/2500 [==============================] - 55s 22ms/step - loss: 0.0184 - accuracy: 0.9953 - val_loss: 0.0705 -

In [ ]:
# model.save("/content/drive/MyDrive/DeepLearningTasks/ GenderClassification/model4.h5")

In [ ]:
test_data_path = '/content/Face Mask Dataset/Test'

data_generator = ImageDataGenerator(rescale = 1/.255)

test_data = data_generator.flow_from_directory(
    test_data_path,
    target_size=(width, height),
    batch_size = batch_size,
    class_mode='categorical')

model.evaluate(test_data)

Found 992 images belonging to 2 classes.
248/248 [==============================] - 3s 13ms/step - loss: 0.1012 - accuracy: 0.9768


[0.10123773664236069, 0.9768145084381104]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[232 251]
 [242 267]]
Classification Report
              precision    recall  f1-score   support

    WithMask       0.49      0.48      0.48       483
 WithoutMask       0.52      0.52      0.52       509

    accuracy                           0.50       992
   macro avg       0.50      0.50      0.50       992
weighted avg       0.50      0.50      0.50       992

